#Intentamos analizar de forma previa los datos

In [1]:
import pandas as pd
import matplotlib as plot

In [3]:
train=pd.read_table('../inputs/train.tsv')
train['len']=train['item_description'].str.len()
train['cant']=1
train.head(20)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,len,cant
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,18.0,1
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,188.0,1
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,124.0,1
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,173.0,1
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,41.0,1
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma...",102.0,1
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,83.0,1
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,268.0,1
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,48.0,1
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0,I realized his pants are on backwards after th...,297.0,1


In [4]:
ag=['name','category_name','brand_name']
t1=train.groupby(ag)['cant'].sum().astype('float').reset_index()
pd1.columns=ag+['count']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['price'].mean().astype('float').reset_index()
t1.columns=ag+['media']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['price'].max().astype('float').reset_index()
t1.columns=ag+['max']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['price'].min().astype('float').reset_index()
t1.columns=ag+['min']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['price'].std().astype('float').reset_index()
t1.columns=ag+['std']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['len'].mean().astype('float').reset_index()
t1.columns=ag+['len_avg']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['len'].min().astype('float').reset_index()
t1.columns=ag+['len_min']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['len'].max().astype('float').reset_index()
t1.columns=ag+['len_max']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
t1=train.groupby(ag)['len'].std().astype('float').reset_index()
t1.columns=ag+['len_std']
pd1=pd1.merge(t1,how='inner',left_on=ag,right_on=ag)
pd1=pd1.set_index(ag)
pd1.head()

NameError: name 'pd1' is not defined

In [ ]:
pdp=pd1[['count']].sort_values(by='count',ascending=False)

In [ ]:
%matplotlib notebook
pdp.plot(kind='line')

In [ ]:
ag=['name','category_name','brand_name']
t1=train.groupby(ag)['cant'].sum().astype('int')
t1.head()

In [ ]:
len(train),train['cant'].sum(),sum(t1)

In [ ]:
len(t1),

In [4]:
ag=['name','category_name','brand_name']
t1=train.groupby(ag,as_index=False)['cant'].sum()

In [ ]:
t1.reindex()

In [6]:
t1['cant'].sum()

846983